In [1]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.postprocess import *
from src.eval import *
gezi.init_flags()

In [2]:

V = 16
models = [
  'deberta-v3-base.len1280.flag-encode_targets-cls-crank.lr-5e-5',
  'deberta-v3-base.len1280.flag-encode_targets-cls-crank.lr-5e-5.a',
  'deberta-v3-base.len1280.flag-encode_targets-cls.lr-5e-5',
  'deberta-v3.len1280.flag-encode_targets-cls-crank',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.ft2',
  'deberta-v3.len1280.flag-encode_targets-cls',
  
  'deberta-v3.len1280.flag-encode_targets-cls.v1_only',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.v1_ft.v1_dynamic',
  'deberta-v3-base.len1280.flag-encode_targets-cls.lr-5e-5.v1_only',
   
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-nl',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-fr',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-de',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-pt',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-af',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cn',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ru',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-fi',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-sv',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ja',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ko',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-el',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-hr',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cy',
  
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cs',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-be',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ar',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-es',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-it',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-da',
]
len(models)

23

In [3]:
oofs = [gezi.read_oof(model, V, return_dict=False) for model in models]

In [4]:
oofs[0]

,text_id,label,n_words,pred,fold
0,0022683E9EA5,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",533,"[2.97, 2.682, 2.98, 2.756, 2.488, 2.639]",0
1,004AC288D833,"[3.5, 4.0, 4.0, 3.5, 3.5, 4.0]",328,"[3.156, 3.076, 3.232, 3.06, 3.098, 3.281]",0
2,01501F95B8B2,"[3.0, 4.0, 3.0, 4.0, 3.5, 3.0]",676,"[3.646, 3.643, 3.754, 3.84, 3.678, 3.434]",0
3,017D2125A1DE,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",379,"[2.38, 2.309, 2.812, 2.375, 2.266, 2.377]",0
4,0230A077F5B6,"[3.0, 2.5, 2.5, 2.5, 2.0, 2.5]",536,"[2.95, 2.63, 2.914, 2.666, 2.432, 2.344]",0
...,...,...,...,...,...
3906,FF7FE6BD8509,"[3.5, 3.5, 3.5, 3.5, 3.5, 3.5]",622,"[3.484, 3.469, 3.621, 3.658, 3.676, 3.281]",4
3907,FFA6690BC309,"[4.0, 3.5, 4.0, 3.5, 3.5, 4.0]",624,"[3.672, 3.271, 3.482, 3.39, 3.09, 3.506]",4
3908,FFAEAF8D0C90,"[2.5, 2.0, 2.5, 1.5, 2.0, 2.0]",496,"[2.758, 2.387, 2.723, 2.354, 2.174, 2.225]",4
3909,FFCDB2524616,"[2.5, 3.0, 3.0, 4.0, 3.5, 3.0]",263,"[3.262, 3.324, 3.477, 3.559, 3.693, 3.344]",4


In [5]:
# svr_oof = oofs[0].copy()

In [6]:
# svr_pred = np.load('../working/svr.npy')
# svr_oof['pred'] = svr_pred

In [7]:
ensembler = gezi.Ensembler(includes=['pred'], inplace=False)
for i, oof in enumerate(oofs):
  if i == 0:
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float))
  elif i == 1: 
    ensembler.add(oof, weight=np.array([1,1,10,1,1,1], float) * 2)
  elif i == 2: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float) * 0.25)
  elif i == 3: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float))
  elif i == 4: 
    ensembler.add(oof, weight=np.array([2,1,10,1,1,2], float) * 2)
  elif i == 5: 
    ensembler.add(oof, weight=np.array([1,1,1,1,10,1], float) * 0.25)
  elif i == 6:
    ensembler.add(oof, weight=np.array([1,1,10,10,10,1], float))
  elif i == 7: 
    ensembler.add(oof, weight=np.array([1,1,10,1,1,1], float))
  elif i == 8: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float) * 0.5)
  else:
    ensembler.add(oof, weight=np.array([1,1,10,1,0.25,0.25], float) * 0.8)

x = ensembler.finalize()
# ensembler2 = gezi.Ensembler(includes=['pred'], inplace=False)
# ensembler2.add(x, weight=np.array([1,1,1,1,1,1], float))
# # ensembler2.add(svr_oof, weight=np.array([1,1,1,1,1,1], float))
# x = ensembler2.finalize()

In [8]:
x['label'] = x.label.apply(list)
x['pred'] = x.pred.apply(list)

In [9]:
m = calc_metrics(np.array(list(x.label.values)), np.array(list(x.pred.values)))
m

OrderedDict([('score', 0.44514639359927893),
             ('score/cohesion', 0.47147124711971033),
             ('score/syntax', 0.4401423006640847),
             ('score/vocabulary', 0.40854637655346476),
             ('score/phraseology', 0.4508095468407472),
             ('score/grammar', 0.46257466576340794),
             ('score/conventions', 0.4373342246542586),
             ('score/cls', 0.46699584509031783),
             ('acc', 0.42482740986959855),
             ('acc/cohesion', 0.3889030938378931),
             ('acc/syntax', 0.42546663257478906),
             ('acc/vocabulary', 0.4622858603937612),
             ('acc/phraseology', 0.4231654308361033),
             ('acc/grammar', 0.4142163129634365),
             ('acc/conventions', 0.43492712861160826),
             ('score/max', 0.5656777675244553),
             ('acc/max', 0.285349015597034),
             ('score/min', 0.3727645085295607),
             ('acc/min', 0.4942469956532856),
             ('score/max2', 0.4447415

In [10]:
calc_metric(np.array(list(x.label.values))[:,0], np.array(list(x.pred.values))[:,0])

0.47147124711971033

In [11]:
import optuna
from optuna import Trial

In [31]:
idx = 9
def deal(i, pseudo=True, min_val=1, max_val=5):
  models = len(oofs)
  target = TARGETS[i]
  ic(i, target, m[f'score/{target}'])
  def objective(trial):
    ensembler = gezi.Ensembler(includes=['pred'], inplace=False)
    weights = [0] * models
    # w = trial.suggest_int(str(idx), min_val, max_val)
    w = trial.suggest_float(str(idx), 0, 1)
    for j in range(models):
      if j < idx:
        if pseudo or (not j in [6, 7, 8]):
          # weights[j] = trial.suggest_int(str(j), min_val, max_val)
          weights[j] = trial.suggest_float(str(j), 0, 1)
      else:
        weights[j] = w
    for j, oof in enumerate(oofs):
      weight = np.array([1,1,1,1,1,1], float)
      weight[i] = weights[j]
      ensembler.add(oof, weight=weight)
    x = ensembler.finalize()
    x['label'] = x.label.apply(list)
    x['pred'] = x.pred.apply(list)
    score = calc_metric(np.array(list(x.label.values))[:,i], np.array(list(x.pred.values))[:,i])
    return score
  study = optuna.create_study(direction='minimize')
  # study.optimize(objective, n_trials=FLAGS.n_trials, callbacks=[wandbc])
  n_trials = 500 if i != 4 else 1000
  study.optimize(objective, n_trials=n_trials)
  return study

In [45]:
study = deal(5, pseudo=False)

[11/30/22 03:13:58] 1284699599.py:5 in deal()
                    i: 5
                    target: 'conventions'
                    m[f'score/{target}']: 0.4373342246542586
[I 2022-11-30 03:13:58,110] A new study created in memory with name: no-name-42eb4d74-3b69-4cd0-a5da-60e483e364ca
[I 2022-11-30 03:13:58,353] Trial 0 finished with value: 0.4381797210896707 and parameters: {'9': 0.1542673436994929, '0': 0.5851528795353996, '1': 0.9471991155389753, '2': 0.797080598963284, '3': 0.7004691345788739, '4': 0.44260277709170814, '5': 0.18308239009318472}. Best is trial 0 with value: 0.4381797210896707.
[I 2022-11-30 03:13:58,590] Trial 1 finished with value: 0.4389021391665421 and parameters: {'9': 0.9808064632101341, '0': 0.737642209732545, '1': 0.5144317690689554, '2': 0.29305313710391145, '3': 0.6315391800439719, '4': 0.7045564071495761, '5': 0.5883865360831686}. Best is trial 0 with value: 0.4381797210896707.
[I 2022-11-30 03:13:58,827] Trial 2 finished with value: 0.4382301596886987 a

In [46]:
idx = 9
l = [0] * (idx + 1)
for i in range(idx + 1):
  if str(i) in study.best_params:
    l[i] = study.best_params[str(i)]
l

[0.05135730870225044,
 0.656092511999863,
 0.0008790512724760023,
 0.17714913118851547,
 0.957691329551472,
 0.04614776691709854,
 0,
 0,
 0,
 0.04985923367555184]

In [47]:
ws = [
  [0.25317958771873733,
 0.04097795038250689,
 0.02655906599013726,
 0.0038228999854169874,
 0.6435230868434522,
 0.0010355910736142909,
 0,
 0,
 0,
 0.10417313001984307],
[0.20584892126103943,
 0.9415947114557675,
 0.0008466019957155957,
 0.034355206308981416,
 0.1943176857454337,
 0.002209925343269196,
 0,
 0,
 0,
 0.13850371052339439],
[0.017133210493372474,
 0.9742527100917538,
 0.08626403044592634,
 0.00020633311365729132,
 0.823078696788037,
 0.0008522631469286782,
 0,
 0,
 0,
 0.4884116649288723],
[0.5751201416404752,
 0.521677447221716,
 0.825191796304036,
 0.36946299169081764,
 0.00015000975639314212,
 0.05593648284142243,
 0,
 0,
 0,
 0.3583963829411852],
[6.87109022382986e-05,
 0.725191709970228,
 0.5221441011946768,
 0.4799948522772268,
 0.36856168008416795,
 0.9639767098301161,
 0,
 0,
 0,
 0.08716018549862706],
[0.05135730870225044,
 0.656092511999863,
 0.0008790512724760023,
 0.17714913118851547,
 0.957691329551472,
 0.04614776691709854,
 0,
 0,
 0,
 0.04985923367555184]
]

In [ ]:
stop

In [ ]:
ws = []
for i in range(len(TARGETS)):
  study = deal(i)
  l = [0] * (idx + 1)
  for i in range(idx + 1):
   l[i] = study.best_params[str(i)]
  ws.append(l)

[11/30/22 02:41:00] 3447548022.py:5 in deal()
                    i: 0
                    target: 'cohesion'
                    m[f'score/{target}']: 0.47147124711971033
[I 2022-11-30 02:41:00,173] A new study created in memory with name: no-name-7f6030ab-a154-429c-b99b-8a51bc2cc6da
[I 2022-11-30 02:41:00,439] Trial 0 finished with value: 0.4718111434317603 and parameters: {'9': 0.8096581825039569, '0': 0.2429862946132294, '1': 0.5767350034302294, '2': 0.5065363041647459, '3': 0.9025294518232088, '4': 0.07830232495771217, '5': 0.6040854867338434, '6': 0.7872286815543378, '7': 0.58596744661445, '8': 0.5772221294031012}. Best is trial 0 with value: 0.4718111434317603.
[I 2022-11-30 02:41:00,679] Trial 1 finished with value: 0.47179678980108136 and parameters: {'9': 0.7972697587864739, '0': 0.021309682417094522, '1': 0.7538927140400411, '2': 0.0480729485857182, '3': 0.6575380613263955, '4': 0.6675978092074017, '5': 0.8647473233888316, '6': 0.01851496798030794, '7': 0.660275815595604, '8

KeyboardInterrupt: 

In [48]:
ws = np.array(ws).transpose()
ws.shape

(10, 6)

In [49]:
ws

array([[2.53179588e-01, 2.05848921e-01, 1.71332105e-02, 5.75120142e-01,
        6.87109022e-05, 5.13573087e-02],
       [4.09779504e-02, 9.41594711e-01, 9.74252710e-01, 5.21677447e-01,
        7.25191710e-01, 6.56092512e-01],
       [2.65590660e-02, 8.46601996e-04, 8.62640304e-02, 8.25191796e-01,
        5.22144101e-01, 8.79051272e-04],
       [3.82289999e-03, 3.43552063e-02, 2.06333114e-04, 3.69462992e-01,
        4.79994852e-01, 1.77149131e-01],
       [6.43523087e-01, 1.94317686e-01, 8.23078697e-01, 1.50009756e-04,
        3.68561680e-01, 9.57691330e-01],
       [1.03559107e-03, 2.20992534e-03, 8.52263147e-04, 5.59364828e-02,
        9.63976710e-01, 4.61477669e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 

In [50]:
idx = 9
ensembler = gezi.Ensembler(includes=['pred'], inplace=False)
for i, oof in enumerate(oofs):
  if i < idx:
    ensembler.add(oof, weight=np.array(ws[i]))
  else:
    ensembler.add(oof, weight=np.array(ws[idx]))

x = ensembler.finalize()

In [51]:
x['label'] = x.label.apply(list)
x['pred'] = x.pred.apply(list)
m = calc_metrics(np.array(list(x.label.values)), np.array(list(x.pred.values)))
m

OrderedDict([('score', 0.445234175591879),
             ('score/cohesion', 0.47128775797153916),
             ('score/syntax', 0.4399263470149584),
             ('score/vocabulary', 0.40867850216795065),
             ('score/phraseology', 0.4510563601260834),
             ('score/grammar', 0.4630637291141399),
             ('score/conventions', 0.4373923571566026),
             ('score/cls', 0.46823821529300075),
             ('acc', 0.4243160317054462),
             ('acc/cohesion', 0.3894144720020455),
             ('acc/syntax', 0.4223983635898747),
             ('acc/vocabulary', 0.46356430580414215),
             ('acc/phraseology', 0.4231654308361033),
             ('acc/grammar', 0.4131935566351317),
             ('acc/conventions', 0.4341600613653797),
             ('score/max', 0.5661098872257675),
             ('acc/max', 0.2873945282536436),
             ('score/min', 0.37303693881385286),
             ('acc/min', 0.4950140628995142),
             ('score/max2', 0.4458687330